In [1]:
import pandas as pd
import requests
from datetime import datetime
import json
import os
from dateutil import parser
import collections

The following code helps to determine whether new data is inserted in api database or not, if so then we have to pull new movies data and put that in database and update tracking parameters. 

In [2]:
last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
first_page = requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=1&limit=50').json()

In [3]:
'movies' in last_page['data'].keys()

True

In [27]:
last_page['data']['movies'][0]['id'], last_page['data']['movies'][-1]['id']

(1494, 1445)

In [28]:
first_page['data']['movies'][0]['id'], first_page['data']['movies'][-1]['id']

(43174, 43126)

In [29]:
ids = [41763,41713]
ids

[41763, 41713]

In [4]:
len(last_page['data']['movies'])

50

In [37]:
def insert_new_movies(ids:list):

    page_number = 1
    new_insertion = 0
    while True:
        init_page = requests.get(f'https://yts.torrentbay.to/api/v2/list_movies.json?page={page_number}&limit=50').json()
        
        current_ids = [init_page['data']['movies'][0]['id'],init_page['data']['movies'][-1]['id']]

        if current_ids > ids:
            new_insertion = new_insertion+len(init_page['data']['movies'])
            for movie in init_page['data']['movies']:
            
        
            
                
            page_number = page_number + 1

        else:
            break

In [38]:
new_movies = check_new_movies(ids)

In [ ]:
def load():
    
    


Below code is for extracting movie cast based on "IMDB code"

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import time

## method to get cast list of given imdb movie id

In [7]:
def get_cast(imdb_id : str):
    
    url = f'https://www.imdb.com/title/{imdb_id}/fullcredits'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'class' : 'cast_list'})
    cast = table.find_all('a')   
    
    return re.findall(r'title="(.*?)"', str(cast))

###  The following data needed to be pulled from api response and put in warehouse
#### : imdb_code -> unique movie code given by IMDB
#### : title_long -> full movie title with year
#### : year -> released year
#### : rating -> movie rating (1-10)
#### : runtime -> in minutes
#### : genres -> can be one or more
#### : summary -> summary of movie
#### : cast -> need to scrape from IMDB
#### : mpa_rating -> movie sensorship rating
#### : language -> can be one or more

In [ ]:
movie_a['imdb_code'], movie_a['title_long'], movie_a['year'], movie_a['rating'], movie_a['runtime'], (movie_a['genres']), movie_a['mpa_rating'], movie_a['language']

Here is the main problem, as movie has multiple genres, we need to figure out how to insert genres without repetation.
First we need to insert "imdb_code" and "movie_title" in movies table. Then we have to check that genre exists in genre table, if so then we can skip, else insert it.

In [8]:
import pyodbc

In [14]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-K8VV6KV;'
                      'Database=yts_warehouse;'
                      'Trusted_Connection=yes;'
                       'autocommit=True')

In [10]:
cursor = conn.cursor()

In [11]:
type(cursor)

pyodbc.Cursor

In [15]:
cursor.execute("""
use yts_warehouse
""");

In [16]:
cursor.execute("""
DBCC CHECKIDENT ('[genre]', RESEED, 0);
""");

In [18]:
cursor.execute("""
DBCC CHECKIDENT ('[cast]', RESEED, 0);
""");

### inserting into movie table

In [ ]:
cursor.execute("""
insert into movies (imdb_id, title, year, rating, runtime, mpa_rating, language, date_uploaded)
VALUES (?,?,?,?,?,?,?,?)
""",
movie_a['imdb_code'], movie_a['title_long'], movie_a['year'],movie_a['rating'], movie_a['runtime'], movie_a['mpa_rating'], movie_a['language'], movie_a['date_uploaded']              
              );

In [28]:
cursor.execute('select * from movies')
for i in cursor:
    print(i)

('tt0386588', 'Hitch (2005)', 2005, 6.6, 118.0, 'PG-13', 'en', datetime.datetime(2015, 10, 31, 23, 58, 31))
('tt0975645', 'Hitchcock (2012)', 2012, 6.8, 98.0, 'PG-13', 'en', datetime.datetime(2015, 10, 31, 23, 58, 36))


### inserting into genre and movie_genre table

In [ ]:
for genre in movie_a['genres']:
    cursor.execute("""
                declare @temp_genre_id int;
                if not exists (
                                select * from genre
                                where genre_title = ?
                              )
                begin
                    insert into genre values(?)
                end
        select @temp_genre_id  = genre_id from genre where genre_title = ?
        insert into movie_genre
        values(?, @temp_genre_id)
    """, genre, genre, genre, movie_a['imdb_code'])

In [29]:
cursor.execute('select *from genre')
for i in cursor:
    print(i)

(1, 'Action')
(2, 'Biography')
(3, 'Comedy')
(4, 'Drama')
(5, 'Romance')


In [30]:
cursor.execute('select *from movie_genre')
for i in cursor:
    print(i)

('tt0975645', 1)
('tt0975645', 2)
('tt0975645', 3)
('tt0975645', 4)
('tt0975645', 5)
('tt0386588', 1)
('tt0386588', 3)
('tt0386588', 5)


In [ ]:
# before that, we need to scrape cast from imdb_page using movie id
cast_list = get_cast(movie_a['imdb_code'])
cast_list

### inserting into cast and movie_cast table

In [ ]:
for actor in cast_list:
    cursor.execute("""
                declare @temp_actor_id int;
                if not exists (
                                select * from cast
                                where actor_name = ?
                              )
                begin
                        insert into cast values(?)
                end
        select @temp_actor_id  = actor_id from cast where actor_name = ?
        insert into movie_cast
        values(?, @temp_actor_id)
    """, actor, actor, actor, movie_a['imdb_code'])

In [31]:
cursor.execute(" select *from cast")
for i in cursor:
    print(i)

(1, 'Anthony Hopkins')
(2, 'Helen Mirren')
(3, 'Scarlett Johansson')
(4, 'Danny Huston')
(5, 'Toni Collette')
(6, 'Michael Stuhlbarg')
(7, 'Michael Wincott')
(8, 'Jessica Biel')
(9, "James D'Arcy")
(10, 'Richard Portnow')
(11, 'Kurtwood Smith')
(12, 'Ralph Macchio')
(13, 'Kai Lennox')
(14, 'Tara Summers')
(15, 'Wallace Langham')
(16, 'Paul Schackman')
(17, 'Currie Graham')
(18, 'Spencer Garrett')
(19, 'Terry Rhoads')
(20, 'Tom Virtue')
(21, 'Karina Deyko')
(22, 'Steven Lee Allen')
(23, 'Richard Chassler')
(24, 'Frank Collison')
(25, 'Melinda Chilton')
(26, 'Mary Anne McGarry')
(27, 'Jon Abrahams')
(28, 'Gil McKinney')
(29, 'Emma Julia Jacobs')
(30, 'Spencer Leigh')
(31, 'Sean MacPherson')
(32, 'Gerald Casale')
(33, 'Tara Arroyave')
(34, 'Judith Hoag')
(35, 'Josh Yeo')
(36, 'Danielle Burgio')
(37, 'John Lacy')
(38, 'Howard Gibson')
(39, 'Josette Prevost')
(40, 'Lauren Stewart')
(41, 'Jaehne Moebius')
(42, 'Ana Matallana')
(43, 'Lindsey Ginter')
(44, 'Paul Henderson')
(45, 'Kay Henderson

In [32]:
cursor.execute(" select *from movie_cast")
for i in cursor:
    print(i)

('tt0975645', 1)
('tt0975645', 2)
('tt0975645', 3)
('tt0975645', 4)
('tt0975645', 5)
('tt0975645', 6)
('tt0975645', 7)
('tt0975645', 8)
('tt0975645', 9)
('tt0975645', 10)
('tt0975645', 11)
('tt0975645', 12)
('tt0975645', 13)
('tt0975645', 14)
('tt0975645', 15)
('tt0975645', 16)
('tt0975645', 17)
('tt0975645', 18)
('tt0975645', 19)
('tt0975645', 20)
('tt0975645', 21)
('tt0975645', 22)
('tt0975645', 23)
('tt0975645', 24)
('tt0975645', 25)
('tt0975645', 26)
('tt0975645', 27)
('tt0975645', 28)
('tt0975645', 29)
('tt0975645', 30)
('tt0975645', 31)
('tt0975645', 32)
('tt0975645', 33)
('tt0975645', 34)
('tt0975645', 35)
('tt0975645', 36)
('tt0975645', 37)
('tt0975645', 38)
('tt0975645', 39)
('tt0975645', 40)
('tt0975645', 41)
('tt0975645', 42)
('tt0975645', 43)
('tt0975645', 44)
('tt0975645', 45)
('tt0975645', 46)
('tt0975645', 47)
('tt0975645', 48)
('tt0975645', 49)
('tt0975645', 50)
('tt0975645', 51)
('tt0975645', 52)
('tt0975645', 53)
('tt0975645', 54)
('tt0975645', 55)
('tt0975645', 56)
(

### inserting into genre and movie_genre table

In [ ]:
movie_a['summary']

In [ ]:
cursor.execute("""

insert into summary (imdb_id, summary)
values (?, ?)

""", movie_a['imdb_code'], movie_a['summary'])

### Method to insert data into rdbms


In [13]:
def load_movie(movie:dict, cursor: pyodbc.Cursor):
    
    # first, we need to extract current movie's cast list from imdb_id
    cast_list = get_cast(movie['imdb_code'])
    
    # now, let's insert data into respective tables one by one 
    
    # 1-> Movie table
    cursor.execute("""
    insert into movies (imdb_id, title, year, rating, runtime, mpa_rating, language, date_uploaded)
    VALUES (?,?,?,?,?,?,?,?)
    """,
    movie['imdb_code'], movie['title_long'], movie['year'],movie['rating'], movie['runtime'], movie['mpa_rating'], movie['language'], movie['date_uploaded']              
                  );
    
    #2-> genre and movie_genre table
    for genre in movie['genres']:
        cursor.execute("""
                    declare @temp_genre_id int;
                    if not exists (
                                    select * from genre
                                    where genre_title = ?
                                  )
                    begin
                        insert into genre values(?)
                    end
            select @temp_genre_id  = genre_id from genre where genre_title = ?
            insert into movie_genre
            values(?, @temp_genre_id)
        """, genre, genre, genre, movie['imdb_code'])
        
    #3-> cast and movie_cast
    for actor in cast_list:
        cursor.execute("""
                    declare @temp_actor_id int;
                    if not exists (
                                    select * from cast
                                    where actor_name = ?
                                  )
                    begin
                            insert into cast values(?)
                    end
            select @temp_actor_id  = actor_id from cast where actor_name = ?
            insert into movie_cast
            values(?, @temp_actor_id)
        """, actor, actor, actor, movie_a['imdb_code'])
    
    cursor.execute("""

        insert into summary (imdb_id, summary)
        values (?, ?)

        """, movie['imdb_code'], movie['summary'])

In [27]:
%%time
load_movie(movie=movie_a, cursor=cursor)

Wall time: 2.41 s


In [33]:
cursor.execute("""
select *from summary
""")

for i in cursor:
    print(i)

('tt0975645', 'In 1959, Sir Alfred Hitchcock (Sir Anthony Hopkins) and his more often than not unheralded collaborator Alma Reville (Dame Helen Mirren), his wife, are revelling in their latest critical and box-office success: North by Northwest (1959). Some in the business believe Sir Alfred, now at age sixty, should hang up his hat while he is still on top, while the brass at Paramount Pictures, to whom he is under contract for one more movie, just wants him to follow North by Northwest (1959) with something in a similar vein to make them money. Sir Alfred, however, wants to stir his creative juices, unlike he has felt he has done with any movie of late. Against the desires of others, including Alma, he is determined that that next project will be a horror movie, specifically an adaptation of a book, that project which will eventually become Psycho (1960). For maximum impact, he wants to buy up whatever existing copies of the book are on the market, so that the public will have no ide